Import Libraries


In [1]:
import boto3 as b3
client = b3.client('ecs')


Configure New Task Defination


In [2]:
task_defination_family = 'Sani-Login'
container_name = 'Sani-Login'
protocol = 'tcp'
container_port = 80
app_protocol = 'http'
network_mode = 'awsvpc'
image_uri = '397102708699.dkr.ecr.ap-south-1.amazonaws.com/upes-colab:login-pg'
execution_role_arn = 'arn:aws:iam::397102708699:role/ecsTaskExecutionRole'
port_name = 'sani-port'

task_definition = {
    'family': task_defination_family,
    'containerDefinitions': [
        {
            'name': container_name,
            'image': image_uri,
            'portMappings': [
                {
                    'containerPort': container_port,
                    'hostPort': container_port,
                    'protocol': protocol,
                    'name': port_name,
                    'appProtocol': app_protocol
                },
            ],
        }
    ],
    'requiresCompatibilities': [
        'FARGATE',
    ],

    'runtimePlatform': {
        'cpuArchitecture': 'X86_64',
        'operatingSystemFamily': 'LINUX'
    },

    'cpu': '256',
    'memory': '512'
}


Create New Task Defination using info provided above


In [3]:
response = client.register_task_definition(
    networkMode=network_mode,
    family=task_definition['family'],
    containerDefinitions=task_definition['containerDefinitions'],
    requiresCompatibilities=task_definition['requiresCompatibilities'],
    runtimePlatform=task_definition['runtimePlatform'],
    cpu=task_definition['cpu'],
    memory=task_definition['memory'],
    executionRoleArn=execution_role_arn
)


Deregister Task Defination


In [4]:
# active_task_def = client.list_task_definitions(
#     familyPrefix = task_defination_family,
#     status='ACTIVE'
# )
# active_task_def
# for i in active_task_def:

res = client.describe_task_definition(
    taskDefinition=task_defination_family
)

result = res['taskDefinition']['revision']

res = client.deregister_task_definition(
    taskDefinition=(task_defination_family + ':' + str(result))
)


Delete Inactive Task Definations


In [5]:
inactive_task_def = client.list_task_definitions(
    familyPrefix=task_defination_family,
    status='INACTIVE'
)
inactive_task_def

res = client.delete_task_definitions(
    taskDefinitions=(inactive_task_def['taskDefinitionArns'])
)
